In [8]:
%%time
import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role
role=get_execution_role()

CPU times: user 76.3 ms, sys: 4.78 ms, total: 81 ms
Wall time: 124 ms


In [15]:
%%time
tfmodel=TensorFlow(entry_point='cats_n_dogs_script.py',
                             role=role,
                             instance_count=1,
                             instance_type='ml.m5.large',
                             framework_version='2.4.1',
                             py_version='py37',
                   script_mode=True,
                  hyperparameters={'epochs': 1})
#                              distribution={'parameter_server': {'enabled': True}})

CPU times: user 133 ms, sys: 12 ms, total: 145 ms
Wall time: 153 ms


In [16]:
%%time
tfmodel.fit(
    {'training':'s3://sagemaker-us-east-1-032934527328/cats_dogs_prediction/',
     'validation':'s3://sagemaker-us-east-1-032934527328/cats_dogs_prediction/'
    })

2021-04-18 12:58:13 Starting - Starting the training job...
2021-04-18 12:58:15 Starting - Launching requested ML instancesProfilerReport-1618750693: InProgress
......
2021-04-18 12:59:28 Starting - Preparing the instances for training......
2021-04-18 13:00:36 Downloading - Downloading input data......
2021-04-18 13:01:33 Training - Downloading the training image..
2021-04-18 13:02:08 Training - Training image download completed. Training in progress.2021-04-18 13:01:50.962320: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-04-18 13:01:50.969272: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-04-18 13:01:51.152365: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-04-18 13:01:56,360 sagemaker-training-toolkit INFO     Imported fr


2021-04-18 13:03:09 Completed - Training job completed
ProfilerReport-1618750693: NoIssuesFound
Training seconds: 135
Billable seconds: 135
CPU times: user 752 ms, sys: 24 ms, total: 776 ms
Wall time: 5min 13s


In [18]:
# Batch transform
tfmodel_predictor=tfmodel.deploy(initial_instance_count=1,instance_type='ml.m5.large')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!

In [26]:
import numpy as np
y=np.random.randint(0, high=255, size=(1,150,150,3), dtype=int)
print(y.shape)

(1, 150, 150, 3)


In [27]:
tfmodel_predictor.predict(y)

{'predictions': [[0.198511407, 0.801488578]]}

In [28]:
tfmodel_predictor.delete_endpoint()